# **응급상황 자동 인식 및 응급실 연계 서비스**
# **단계3 : 응급상황 연계(추천)**

## **0.미션**

단계 3에서는, 응급상황의 음성을 인식해서 텍스트로 변환하고, 변환된 텍스트를 다시 요약 및 핵심키워드 도출 작업을 수행합니다.  
이를 위해 사전학습된 모델을 API로 연결하여 활용합니다.

### 미션4 : 응급실 추천
* 응급실 위치와 응급전화 발신자 위치 기반 추천
* 두 좌표간 직선거리(Haversine)
    * 1) 500여 곳 응급실에 대해서, 거리 기반 가까운 응급실 찾기
    * 2) 좌표 구간을 설정하여 대상 응급실 범위를 좁힌 후, 거리 기반 가까운 응급실 찾기


## **1.환경설정**

### (1) 경로 설정

구글 드라이브 연결

#### 1) 구글 드라이브 폴더 생성
* 새 폴더(project6_2)를 생성하고
* 제공 받은 파일을 업로드

#### 2) 구글 드라이브 연결

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
path = '/content/drive/MyDrive/kt/projects_kt/project_6_2/6_2/'

### (2) 라이브러리

#### 1) 필요한 라이브러리 설치

* requirements.txt 파일의 [경로 복사]를 한 후,
* 아래 경로에 붙여 넣기

In [5]:
# 경로 : /content/drive/MyDrive/project6_2/requirements.txt
# 경로가 다른 경우 아래 코드의 경로 부분을 수정하세요.

!pip install -r /content/drive/MyDrive/kt/projects_kt/project_6_2/6_2/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


#### 2) 라이브러리 로딩

In [6]:
#필요한 라이브러리 설치 및 불러우기
import os
import pandas as pd
import numpy as np

from haversine import haversine
import requests
import json

# 더 필요한 라이브러리 추가 -------------




### (3) 데이터 로딩
* 단계1에서 수집한 응급실 정보를 불러와서 데이터프레임으로 저장합니다.

In [11]:
data = pd.read_csv(path+'응급실 정보.csv')
data.head()

,병원이름,주소,응급의료기관 종류,전화번호 1,전화번호 3,위도,경도
0,(의)내경의료재단울산제일병원,울산광역시 남구 남산로354번길 26 (신정동),응급실운영신고기관,052-220-3300,052-220-3334,35.548238,129.307011
1,(의)서일의료재단기장병원,부산광역시 기장군 기장읍 대청로72번길 6,지역응급의료기관,051-723-0171,051-723-2119,35.236029,129.216492
2,(의)성세의료재단 뉴성민병원,"인천광역시 서구 칠천왕로33번길 17 (석남동, 신석로 70(석남1동, 성민병원))",지역응급의료기관,032-726-1000,032-726-1190,37.508994,126.669479
3,(의)영문의료재단다보스병원,"경기도 용인시 처인구 백옥대로1082번길 18, 다보스종합병원 (김량장동)",지역응급의료센터,031-8021-2114,031-8021-2130,37.234641,127.210499
4,(의)효심의료재단용인서울병원,경기도 용인시 처인구 고림로 81 (고림동),지역응급의료기관,031-337-0114,031-336-0119,37.240316,127.214491


## **2. 응급실 추천**


### (1) 직선거리 계산
- haversine formula
    * Haversine은 두 지점 간의 거리를 구할 때 사용하는 수학 공식으로, 지구의 구형 구조를 고려하여 위도와 경도를 기반으로 직선 거리를 계산한다.
- 세부사항
    * 하버사인 함수를 활용


#### 1) 하버사인 함수 사용 연습
* 임의의 두 좌표간 거리 계산
    * 응급실 데이터프레임을 열어서
    * 응급실 두 곳의 좌표를 확인하고
    * 두 지점의 거리를 계산해 봅시다
* 사용법 : haversine((위도1, 경도1), (위도2, 경도2), unit='km')


In [ ]:
haversine((),(),unit='km')

#### 2) 가장 가까운 응급실 3곳 추천하기1
* 세부사항
    * 입력된 좌표와 전체 응급실과 거리를 계산한 후
    * 가장 가까운 거리의 응급실 3 곳을 반환합니다.
* 이를 함수로 생성하고 테스트 해 봅시다.

In [19]:
from re import I
box = []
current_location = (37.5159,127.131)
for i in range(len(data)):
    box.append({'name':data['병원이름'][i],'address':data['주소'][i],'distance':haversine((current_location[0],current_location[1]),(data['위도'][i],data['경도'][i]),unit='km')})
box.sort(key=lambda x: x['distance'])
box = pd.DataFrame(box)
box.head(3)

,name,address,distance
0,한국보훈복지의료공단중앙보훈병원,서울특별시 강동구 진황도로61길 53 (둔촌동),1.949021
1,성심의료재단강동성심병원,서울특별시 강동구 성안로 150 (길동),2.264680
2,경찰병원,"서울특별시 송파구 송이로 123, 국립경찰병원 (가락동)",2.265891


In [21]:
def recommend_hospital1(lat, lng):
    box = []
    current_location = (lat,lng)
    for i in range(len(data)):
        box.append({'name':data['병원이름'][i],'address':data['주소'][i],'distance':haversine((current_location[0],current_location[1]),(data['위도'][i],data['경도'][i]),unit='km')})
    box.sort(key=lambda x: x['distance'])
    box = pd.DataFrame(box)
    return box.head(3)

recommend_hospital1(37.5159,127.131)

,name,address,distance
0,한국보훈복지의료공단중앙보훈병원,서울특별시 강동구 진황도로61길 53 (둔촌동),1.949021
1,성심의료재단강동성심병원,서울특별시 강동구 성안로 150 (길동),2.264680
2,경찰병원,"서울특별시 송파구 송이로 123, 국립경찰병원 (가락동)",2.265891


#### 3) 가장 가까운 응급실 3곳 추천하기2
* 문제점 : 입력 받은 좌표와 응급실 전체와의 거리를 모두 계산하는 것은 비효율 적입니다.
* 해결 방안 : 그래서 입력 받은 좌표를 기준으로 일정 범위 내에 해당되는 응급실에 대해서 거리를 계산하고 추천하도록 기존 함수를 수정 합니다.
* hint :
    * 입력 받은 위도, 경도 값에 ± α 하여 일정 범위 구간을 정하고
    * 응급실 정보에서 해당 구간을 먼저 조회한 후
    * 거리 계산

In [22]:


def recommend_hospital2(lat, lng, alpha=0.1):

    min_lat = lat - alpha
    max_lat = lat + alpha
    min_lng = lng - alpha
    max_lng = lng + alpha

    # 범위 내 응급실 필터링
    target_hospitals = data[(data['위도'] >= min_lat) & (data['위도'] <= max_lat) &
                           (data['경도'] >= min_lng) & (data['경도'] <= max_lng)]

    box = []
    current_location = (lat, lng)
    for i in target_hospitals.index:
        box.append({'name': target_hospitals['병원이름'][i],
                    'address': target_hospitals['주소'][i],
                    'distance': haversine((current_location[0], current_location[1]),
                                         (target_hospitals['위도'][i], target_hospitals['경도'][i]), unit='km')})

    box.sort(key=lambda x: x['distance'])
    box = pd.DataFrame(box)
    return box.head(3)

# 예시 사용
recommend_hospital2(37.5159, 127.131)

,name,address,distance
0,한국보훈복지의료공단중앙보훈병원,서울특별시 강동구 진황도로61길 53 (둔촌동),1.949021
1,성심의료재단강동성심병원,서울특별시 강동구 성안로 150 (길동),2.264680
2,경찰병원,"서울특별시 송파구 송이로 123, 국립경찰병원 (가락동)",2.265891


### (2) [조 과제]고도화 : naver 지도 api 사용

* 이 부분은 조별 과제로 수행하게 됩니다.(개인과제 아님!)

* 세부사항
    * 두 지점간, 최단 도로거리, 소요 시간을 계산하는 함수를 생성하시오.
    * 함수 내용
        * 입력 : 두 지점의 위도, 경도, 네이버클라우드id, 암호키
        * 출력 : 도로거리(km)
    
    * 네이버 Maps API 활용
        * 사용할 API : Direction 5
        * 가이드 : https://guide.ncloud-docs.com/docs/ko/maps-direction5-api
        * 가이드를 활용해서 url, header, params를 구성합니다.
        * params의 옵션은 'trafast' (실시간 빠른 길 옵션)을 선택하시오.

#### 1) maps 클라이언트ID, 키 로딩

In [27]:
c_id = 'sb7vih3g35'
c_key = '4hp1aMpvQ9mRmhQ6h9U1xd5hV96x2so6Iz74vYiU'

#### 2) 함수 생성

In [29]:
def get_dist(start_lat, start_lng, dest_lat, dest_lng):
    url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": 'sb7vih3g35',
        "X-NCP-APIGW-API-KEY": '4hp1aMpvQ9mRmhQ6h9U1xd5hV96x2so6Iz74vYiU',
    }
    params = {
        "start": f"{start_lng},{start_lat}",  # 출발지 (경도, 위도)
        "goal": f"{dest_lng},{dest_lat}",    # 목적지 (경도, 위도)
        "option": "trafast"  # 실시간 빠른 길 옵션
    }

    # 요청하고, 답변 받아오기
    response = requests.get(url, headers = headers , params = params)


    if response.status_code == 200:
        data = response.json()  # JSON 응답을 파싱
        try:
            dist = data['route']['trafast'][0]['summary']['distance']  # 거리(m)
            return dist
        except (KeyError, IndexError):
            raise ValueError("올바르지 않은 응답 데이터 형식입니다.")
    else:
        raise ConnectionError(f"API 요청 실패: {response.status_code}, {response.text}")



* 테스트

In [31]:
start_lat = 37.5665  # 예시: 서울 위도
start_lng = 126.9780  # 예시: 서울 경도
dest_lat = 37.4513  # 예시: 성남 위도
dest_lng = 127.1370  # 예시: 성남 경도
try:
    distance = get_dist(start_lat, start_lng, dest_lat, dest_lng)
    print(f"거리: {distance} m")
except Exception as e:
    print(f"에러 발생: {e}")

에러 발생: get_dist() missing 2 required positional arguments: 'c_id' and 'c_key'


#### 3) 응급실 추천
* recommend_hospital2 함수를 참조해서 recommend_hospital3 만들기
    * 거리 계산 부분을 get_dist 함수로 대체
    * 입력 부분 수정

In [32]:


def recommend_hospital3(lat, lng, alpha=0.1):

    min_lat = lat - alpha
    max_lat = lat + alpha
    min_lng = lng - alpha
    max_lng = lng + alpha

    # 범위 내 응급실 필터링
    target_hospitals = data[(data['위도'] >= min_lat) & (data['위도'] <= max_lat) &
                           (data['경도'] >= min_lng) & (data['경도'] <= max_lng)]

    box = []
    current_location = (lat, lng)
    for i in target_hospitals.index:
        box.append({'name': target_hospitals['병원이름'][i],
                    'address': target_hospitals['주소'][i],
                    'distance': get_dist(current_location[0],current_location[1],
                                         data['위도'][i],data['경도'][i])})

    box.sort(key=lambda x: x['distance'])
    box = pd.DataFrame(box)
    return box.head(3)
2
# 예시 사용
recommend_hospital2(37.5159, 127.131)

,name,address,distance
0,한국보훈복지의료공단중앙보훈병원,서울특별시 강동구 진황도로61길 53 (둔촌동),1.949021
1,성심의료재단강동성심병원,서울특별시 강동구 성안로 150 (길동),2.264680
2,경찰병원,"서울특별시 송파구 송이로 123, 국립경찰병원 (가락동)",2.265891


## **Mission Complete!**

수고 많았습니다!